# "2020 January One Day Holding Long Short Portfolio BackTest"
> "2020 January Portfolio"
- toc: false
- branch: master
- badges: false
- comments: true
- categories: [long, short, longshort]
- hide: false
- search_exclude: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2

In [17]:
#hide_input
import numpy as np
import pandas as pd
import seaborn as sns
import os
import time
import gc
import glob
import matplotlib.pyplot as plt

In [18]:
#hide_input
df_testp=pd.read_csv('D:\\Downloads\\20204dayreturnpred2019train60K.csv')
df_test=pd.read_csv('D:\\Downloads\\8K20194dayreturnpred2019train60K.csv')

In [19]:
#hide
df_testp=df_testp.loc[df_testp.label!='nan'].loc[df_testp.Date>=20200101].loc[df_testp.Date<20200201].sort_values(by='Date',ascending=True).copy()
df_test=df_test.loc[df_test.label!='nan'].loc[df_test.Date>=20200101].loc[df_test.Date<20200201].sort_values(by='Date',ascending=True).copy()
#df_testp.reset_index(drop=True, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


**Long-Short Portfolio**

In [20]:
#hide
df_agg=df_testp[['Date','pred','labelr1']].groupby(['Date','pred']).sum().reset_index();
df_agg['cumreturn1']=0.0
for l in range(len(df_agg)):
    df_agg['cumreturn1'][l]=np.prod((1+(df_agg['labelr1']*df_agg['pred']/100))[:l+1])
    
#hide
df_ag=df_test[['Date','pred','labelr1']].groupby(['Date','pred']).sum().reset_index();
df_ag['cumreturn1a']=0.0
for l in range(len(df_ag)):
    df_ag['cumreturn1a'][l]=np.prod((1+(df_ag['labelr1']*df_ag['pred']/100))[:l+1])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [21]:
#hide
if len(df_agg)>0:
    import altair as alt
    import datetime
    df_agg['Date']=df_agg['Date'].astype(str).str[:4]+'-'+df_agg['Date'].astype(str).str[4:6]+'-'+df_agg['Date'].astype(str).str[6:]
    df_agg['Date']=df_agg['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=pd.read_csv('D:\\Downloads\\GSPC.csv')
    df_dow['Date']=df_dow['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=df_dow.loc[df_dow.Date>=df_agg.Date.min()].loc[df_dow.Date<=df_agg.Date.max()].reset_index()
    df_dow['return']=((df_dow['Close'].shift(-1)-df_dow['Close'])/df_dow["Close"])
    df_dow['cumreturndow']=0.0
    for l in range(len(df_dow)):
        df_dow['cumreturndow'][l]=np.prod((1+(df_dow['return']))[:l+1])
    df_agg=pd.merge(df_agg,df_dow[['Date','cumreturndow']], right_on='Date', left_on='Date',how='inner')
    df_sp500=pd.read_csv('D:\\Downloads\\DJI.csv')
    df_sp500['Date']=df_sp500['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_sp500=df_sp500.loc[df_sp500.Date>=df_agg.Date.min()].loc[df_sp500.Date<=df_agg.Date.max()].reset_index()
    df_sp500['return']=((df_sp500['Close'].shift(-1)-df_sp500['Close'])/df_sp500["Close"])
    df_sp500['cumreturnsp500']=0.0
    for l in range(len(df_sp500)):
        df_sp500['cumreturnsp500'][l]=np.prod((1+(df_sp500['return']))[:l+1])
    
    df_agg=pd.merge(df_agg,df_sp500[['Date','cumreturnsp500']], right_on='Date', left_on='Date',how='inner')
    df_ag['Date']=df_ag['Date'].astype(str).str[:4]+'-'+df_ag['Date'].astype(str).str[4:6]+'-'+df_ag['Date'].astype(str).str[6:]
    df_ag['Date']=df_ag['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_agg=pd.merge(df_agg,df_ag[['Date','cumreturn1a']], right_on='Date', left_on='Date',how='inner')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [22]:
#hide_input
df_agg

,Date,pred,labelr1,cumreturn1,cumreturndow,cumreturnsp500,cumreturn1a
0,2020-01-02,1.0,7.058824,1.070588,0.992940,0.991897,1.524213
1,2020-01-02,1.0,7.058824,1.070588,0.992940,0.991897,1.492427
2,2020-01-03,-1.0,-2.409297,1.096382,0.996449,0.994270,1.169006
3,2020-01-03,-1.0,-2.409297,1.096382,0.996449,0.994270,1.034547
4,2020-01-07,-1.0,0.000000,1.096382,0.998527,0.995715,1.638976
5,2020-01-07,-1.0,0.000000,1.096382,0.998527,0.995715,1.706580
6,2020-01-07,1.0,0.000000,1.096382,0.998527,0.995715,1.638976
7,2020-01-07,1.0,0.000000,1.096382,0.998527,0.995715,1.706580
8,2020-01-10,-1.0,1.048658,1.084885,1.009294,1.001325,3.038341
9,2020-01-10,-1.0,1.048658,1.084885,1.009294,1.001325,3.355847


In [23]:
#hide_input
import altair as alt
a1 = alt.Chart(df_agg).mark_trail(color='steelblue').encode(
    x='Date:T',
    y='cumreturn1:Q',
    size='cumreturn1',
    
).properties(
    width=1000,
    height=1000,
    title='10KPortfolio (blue) vs 8KPortfolio (brown) vs  Dow (black) vs SP500 (red)'
)

a2 = alt.Chart(df_agg).mark_circle(color='black',size=40).encode(
    x='Date:T',
    y='cumreturndow:Q',

).properties(
    width=1000,
    height=1000
)

a3 = alt.Chart(df_agg).mark_line(color='red').encode(
    x='Date:T',
    y='cumreturnsp500:Q',

).properties(
    width=1000,
    height=1000
)

a4 = alt.Chart(df_agg).mark_line(color='brown').encode(
    x='Date:T',
    y='cumreturn1a:Q',

).properties(
    width=1000,
    height=1000
)

a1 + a2 + a3 + a4

alt.LayerChart(...)

**Long portfolio**

In [24]:
#hide
df_agg=df_testp[['Date','pred','labelr1']].loc[df_testp.pred==1.0].groupby(['Date','pred']).sum().reset_index();
df_agg['cumreturn1']=0.0
for l in range(len(df_agg)):
    df_agg['cumreturn1'][l]=np.prod((1+(df_agg['labelr1']*df_agg['pred']/100))[:l+1])

df_ag=df_test[['Date','pred','labelr1']].loc[df_test.pred==1.0].groupby(['Date','pred']).sum().reset_index();
df_ag['cumreturn1a']=0.0
for l in range(len(df_ag)):
    df_ag['cumreturn1a'][l]=np.prod((1+(df_ag['labelr1']*df_ag['pred']/100))[:l+1])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [25]:
#hide
if len(df_agg)>0:
    import altair as alt
    import datetime
    df_agg['Date']=df_agg['Date'].astype(str).str[:4]+'-'+df_agg['Date'].astype(str).str[4:6]+'-'+df_agg['Date'].astype(str).str[6:]
    df_agg['Date']=df_agg['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=pd.read_csv('D:\\Downloads\\GSPC.csv')
    df_dow['Date']=df_dow['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=df_dow.loc[df_dow.Date>=df_agg.Date.min()].loc[df_dow.Date<=df_agg.Date.max()].reset_index()
    df_dow['return']=((df_dow['Close'].shift(-1)-df_dow['Close'])/df_dow["Close"])
    df_dow['cumreturndow']=0.0
    for l in range(len(df_dow)):
        df_dow['cumreturndow'][l]=np.prod((1+(df_dow['return']))[:l+1])
    df_agg=pd.merge(df_agg,df_dow[['Date','cumreturndow']], right_on='Date', left_on='Date',how='inner')
    df_sp500=pd.read_csv('D:\\Downloads\\DJI.csv')
    df_sp500['Date']=df_sp500['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_sp500=df_sp500.loc[df_sp500.Date>=df_agg.Date.min()].loc[df_sp500.Date<=df_agg.Date.max()].reset_index()
    df_sp500['return']=((df_sp500['Close'].shift(-1)-df_sp500['Close'])/df_sp500["Close"])
    df_sp500['cumreturnsp500']=0.0
    for l in range(len(df_sp500)):
        df_sp500['cumreturnsp500'][l]=np.prod((1+(df_sp500['return']))[:l+1])
    df_agg=pd.merge(df_agg,df_sp500[['Date','cumreturnsp500']], right_on='Date', left_on='Date',how='inner')
    df_ag['Date']=df_ag['Date'].astype(str).str[:4]+'-'+df_ag['Date'].astype(str).str[4:6]+'-'+df_ag['Date'].astype(str).str[6:]
    df_ag['Date']=df_ag['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_agg=pd.merge(df_agg,df_ag[['Date','cumreturn1a']], right_on='Date', left_on='Date',how='inner')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [26]:
#hide_input
df_agg

,Date,pred,labelr1,cumreturn1,cumreturndow,cumreturnsp500,cumreturn1a
0,2020-01-02,1.0,7.058824,1.070588,0.992940,0.991897,0.979146
1,2020-01-07,1.0,0.000000,1.070588,0.998527,0.995715,0.867216
2,2020-01-14,1.0,0.000000,1.070588,1.009651,1.005592,2.147692
3,2020-01-16,1.0,-8.333333,0.981373,1.022030,1.016603,2.828644
4,2020-01-21,1.0,0.000000,0.981373,1.019614,1.010997,1.726124
5,2020-01-23,1.0,-7.480315,0.907963,1.011547,1.004189,1.390103
6,2020-01-24,1.0,0.042355,0.908347,0.995635,0.988465,1.002301
7,2020-01-27,1.0,0.303819,0.911107,1.005645,0.994944,0.868850
8,2020-01-28,1.0,-2.812500,0.885482,1.004773,0.995346,0.572719
9,2020-01-29,1.0,-9.059422,0.805263,1.007922,0.999676,0.554732


In [27]:
#hide_input
a1 = alt.Chart(df_agg).mark_trail(color='steelblue').encode(
    x='Date:T',
    y='cumreturn1:Q',
    size='cumreturn1',
    
).properties(
    width=1000,
    height=1000,
    title='10KPortfolio (blue) vs 8KPortfolio (brown) vs  Dow (black) vs SP500 (red)'
)

a2 = alt.Chart(df_agg).mark_circle(color='black',size=40).encode(
    x='Date:T',
    y='cumreturndow:Q',

).properties(
    width=1000,
    height=1000
)

a3 = alt.Chart(df_agg).mark_line(color='red').encode(
    x='Date:T',
    y='cumreturnsp500:Q',

).properties(
    width=1000,
    height=1000
)

a4 = alt.Chart(df_agg).mark_line(color='brown').encode(
    x='Date:T',
    y='cumreturn1a:Q',

).properties(
    width=1000,
    height=1000
)

a1 + a2 + a3 + a4 

alt.LayerChart(...)

**Short Portfolio**

In [28]:
#hide
df_agg=df_testp[['Date','pred','labelr1']].loc[df_testp.pred==-1.0].groupby(['Date','pred']).sum().reset_index();
df_agg['cumreturn1']=0.0
for l in range(len(df_agg)):
    df_agg['cumreturn1'][l]=np.prod((1+(df_agg['labelr1']*df_agg['pred']/100))[:l+1])

df_ag=df_test[['Date','pred','labelr1']].loc[df_test.pred==-1.0].groupby(['Date','pred']).sum().reset_index();
df_ag['cumreturn1a']=0.0
for l in range(len(df_ag)):
    df_ag['cumreturn1a'][l]=np.prod((1+(df_ag['labelr1']*df_ag['pred']/100))[:l+1])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [29]:
#hide
if len(df_agg)>0:
    import altair as alt
    import datetime
    df_agg['Date']=df_agg['Date'].astype(str).str[:4]+'-'+df_agg['Date'].astype(str).str[4:6]+'-'+df_agg['Date'].astype(str).str[6:]
    df_agg['Date']=df_agg['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=pd.read_csv('D:\\Downloads\\GSPC.csv')
    df_dow['Date']=df_dow['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=df_dow.loc[df_dow.Date>=df_agg.Date.min()].loc[df_dow.Date<=df_agg.Date.max()].reset_index()
    df_dow['return']=((df_dow['Close'].shift(-1)-df_dow['Close'])/df_dow["Close"])
    df_dow['cumreturndow']=0.0
    for l in range(len(df_dow)):
        df_dow['cumreturndow'][l]=np.prod((1+(df_dow['return']))[:l+1])
    df_agg=pd.merge(df_agg,df_dow[['Date','cumreturndow']], right_on='Date', left_on='Date',how='inner')
    df_sp500=pd.read_csv('D:\\Downloads\\DJI.csv')
    df_sp500['Date']=df_sp500['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_sp500=df_sp500.loc[df_sp500.Date>=df_agg.Date.min()].loc[df_sp500.Date<=df_agg.Date.max()].reset_index()
    df_sp500['return']=((df_sp500['Close'].shift(-1)-df_sp500['Close'])/df_sp500["Close"])
    df_sp500['cumreturnsp500']=0.0
    for l in range(len(df_sp500)):
        df_sp500['cumreturnsp500'][l]=np.prod((1+(df_sp500['return']))[:l+1])
    df_agg=pd.merge(df_agg,df_sp500[['Date','cumreturnsp500']], right_on='Date', left_on='Date',how='inner')
    df_ag['Date']=df_ag['Date'].astype(str).str[:4]+'-'+df_ag['Date'].astype(str).str[4:6]+'-'+df_ag['Date'].astype(str).str[6:]
    df_ag['Date']=df_ag['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_agg=pd.merge(df_agg,df_ag[['Date','cumreturn1a']], right_on='Date', left_on='Date',how='inner')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [30]:
#hide_input
df_agg

,Date,pred,labelr1,cumreturn1,cumreturndow,cumreturnsp500,cumreturn1a
0,2020-01-03,-1.0,-2.409297,1.024093,1.003533,1.002392,1.193904
1,2020-01-07,-1.0,0.000000,1.024093,1.005626,1.003849,1.967884
2,2020-01-10,-1.0,1.048658,1.013354,1.016471,1.009505,3.371352
3,2020-01-13,-1.0,0.000000,1.013354,1.014931,1.010644,0.770777
4,2020-01-14,-1.0,-4.021414,1.054105,1.016829,1.013806,0.680454
5,2020-01-21,-1.0,0.017143,1.053924,1.026864,1.019256,0.999499
6,2020-01-22,-1.0,-17.226219,1.235475,1.028035,1.018342,1.013759
7,2020-01-24,-1.0,-1.588535,1.255101,1.002714,0.996540,3.354604
8,2020-01-27,-1.0,0.728191,1.245962,1.012795,1.003072,5.645585
9,2020-01-28,-1.0,50.675722,0.614562,1.011917,1.003477,5.161384


In [31]:
#hide_input
a1 = alt.Chart(df_agg).mark_trail(color='steelblue').encode(
    x='Date:T',
    y='cumreturn1:Q',
    size='cumreturn1',
    
).properties(
    width=1000,
    height=1000,
    title='Portfolio (blue) vs Dow (black) vs SP500 (red)'
)

a2 = alt.Chart(df_agg).mark_circle(color='black',size=40).encode(
    x='Date:T',
    y='cumreturndow:Q',

).properties(
    width=1000,
    height=1000
)

a3 = alt.Chart(df_agg).mark_line(color='red').encode(
    x='Date:T',
    y='cumreturnsp500:Q',

).properties(
    width=1000,
    height=1000
)

a4 = alt.Chart(df_agg).mark_line(color='brown').encode(
    x='Date:T',
    y='cumreturn1a:Q',

).properties(
    width=1000,
    height=1000
)

a1 + a2 + a3 + a4  

alt.LayerChart(...)